In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np

In [2]:
# Загрузка данных: пропускаем первую строку с заголовками
data = pd.read_csv('/Users/umanindaniil/KAN/datasets_raw/db_nl_preprocessed-edit.csv')
data = data.dropna()
feature_names = list(data.columns[:-1])

X = data.iloc[:, :-1].values  # Признаки
y = data.iloc[:, -1].values   # Целевая переменная

# Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Стандартизация данных
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Преобразование в тензоры
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [3]:
# Импорт модели KAN из официальной реализации pykan
# Если импорт не проходит, попробуйте изменить путь, например:
# from pykan.model import KAN
from kan import KAN

dims = [X_train.shape[1], 64, 32, 1]
model = KAN(dims)

# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()  # Для задачи регрессии; для классификации используйте другую функцию потерь
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Обучение модели
epochs = 100
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}')

# Оценка на тестовой выборке
model.eval()
with torch.no_grad():
    predictions = model(X_test_tensor)
    test_loss = criterion(predictions, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')


checkpoint directory created: ./model
saving model version 0.0
Epoch 10/100, Loss: 0.2713
Epoch 20/100, Loss: 0.2186
Epoch 30/100, Loss: 0.2082
Epoch 40/100, Loss: 0.1930
Epoch 50/100, Loss: 0.1825
Epoch 60/100, Loss: 0.1744
Epoch 70/100, Loss: 0.1677
Epoch 80/100, Loss: 0.1623
Epoch 90/100, Loss: 0.1579
Epoch 100/100, Loss: 0.1543
Test Loss: 0.1753


In [4]:
data

,номер,возраст,пол,масса_тела,рост,холестерин,нас._жир,натрий_na,сахара,энергия,...,йод_i,медь_cu,селен_se,цинк_zn,профессия_работники_преимущественно_умственного_труда,профессия_работники_занятые_легким_физическим_трудом,спорт_легкий_спорт,спорт_не_занимаюсь,bmi,target
1,84,23,0,75.0,180.0,200.19,11.73,906.53,4.24,536.60,...,18.93,0.26,57.96,5.46,1,0,0,1,23.148148,0
2,86,41,0,96.0,177.0,450.90,40.44,1750.81,90.92,1470.00,...,0.60,0.04,17.46,2.03,1,0,0,1,30.642536,1
4,116,21,0,77.0,171.0,58.50,12.74,642.27,2.55,306.93,...,4.28,0.05,9.90,0.81,1,0,1,0,26.332889,0
5,118,25,1,40.0,158.0,123.19,27.19,493.94,161.37,1554.10,...,6.28,0.26,56.36,4.63,1,0,1,0,16.023073,0
6,146,19,0,58.0,170.0,398.98,27.38,1069.96,10.74,767.73,...,50.22,0.18,23.06,2.88,1,0,0,0,20.069204,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1957,2881,44,0,104.0,170.0,477.17,24.50,1797.91,82.62,1354.90,...,113.14,0.73,60.15,4.58,1,0,0,1,35.986159,0
1958,2882,60,0,180.2,174.0,1867.95,34.70,3373.56,81.50,2929.34,...,372.68,2.34,254.15,24.05,1,0,0,1,59.519091,1
1959,2883,47,1,82.0,162.0,356.44,33.42,3321.43,115.59,2744.22,...,251.33,2.12,146.40,12.77,1,0,0,1,31.245237,1
1960,2884,45,0,174.8,185.0,1130.65,61.98,5649.49,114.94,3263.85,...,121.06,1.86,199.63,17.86,1,0,0,1,51.073776,1


In [5]:
# Предсказание для одного примера (например, первый элемент тестовой выборки)
sample = X_test_tensor[0:1]
prediction = model(sample)
print("Предсказание:", prediction.item())

Предсказание: 0.7748001217842102


/Users/umanindaniil/KAN/.venv/lib/python3.9/site-packages/kan/MultKAN.py:813: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  self.subnode_actscale.append(torch.std(x, dim=0).detach())
/Users/umanindaniil/KAN/.venv/lib/python3.9/site-packages/kan/MultKAN.py:823: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  input_range = torch.std(preacts, dim=0) + 0.1
/Users/umanindaniil/KAN/.venv/lib/python3.9/site-packages/kan/MultKAN.py:824: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output n

In [6]:
# Включаем вычисление градиентов для входного примера
sample = X_test_tensor[0:1]
sample.requires_grad = True

# Получаем предсказание
prediction = model(sample)
# Выполняем обратное распространение ошибки
prediction.backward()

# Градиенты по входным признакам отражают, насколько изменение каждого признака повлияет на предсказание
print("Градиенты по входным признакам (интерпретация):")
print(sample.grad)


Градиенты по входным признакам (интерпретация):
tensor([[-0.0343,  0.2365,  0.0033,  0.0697, -0.0328, -0.0324,  0.0107, -0.0005,
         -0.0113, -0.0167, -0.0038, -0.0080, -0.0062,  0.0039,  0.0198, -0.0078,
         -0.0122,  0.0016, -0.0073, -0.0155,  0.0032, -0.0103, -0.0100,  0.0182,
         -0.0012, -0.0190, -0.0123, -0.0039,  0.0503, -0.0130, -0.0068,  0.0015,
         -0.0037, -0.0361, -0.0200,  0.0143,  0.0254,  0.0168, -0.0061, -0.0037,
          0.0267,  0.0251, -0.0394, -0.0022,  0.0095,  0.0217,  0.0599]])


In [7]:
import numpy as np
import pandas as pd

# Читаем датасет с заголовками (первая строка – названия признаков)
data = pd.read_csv('../datasets_raw/db_nl_preprocessed-edit.csv')

# Извлекаем признаки и целевую переменную
X = data.iloc[:, :-1].values  # Признаки
y = data.iloc[:, -1].values   # Целевая переменная

# Сохраняем читаемые названия признаков (все столбцы, кроме последнего)
feature_names = list(data.columns[:-1])

# Разделение данных, масштабирование и т.д.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

# Предположим, что модель уже обучена и находится в переменной model

# Получаем интерпретацию для одного примера из тестовой выборки:
sample = X_test_tensor[0:1]
sample.requires_grad = True

# Получаем предсказание и вычисляем градиенты
prediction = model(sample)
prediction.backward()

# Получаем градиенты в виде numpy массива
gradients = sample.grad.detach().numpy().flatten()

# Вычисляем абсолютные значения градиентов для оценки важности признаков
abs_gradients = np.abs(gradients)
total_importance = np.sum(abs_gradients)
importance_percent = (abs_gradients / total_importance) * 100

# Формируем DataFrame, где в первой колонке читаемые названия признаков, а во второй — их важность в процентах
df_importance = pd.DataFrame({
    "Признак": feature_names,
    "Важность (%)": importance_percent
})

print(df_importance)


/Users/umanindaniil/KAN/.venv/lib/python3.9/site-packages/kan/MultKAN.py:813: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  self.subnode_actscale.append(torch.std(x, dim=0).detach())
/Users/umanindaniil/KAN/.venv/lib/python3.9/site-packages/kan/MultKAN.py:823: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  input_range = torch.std(preacts, dim=0) + 0.1
/Users/umanindaniil/KAN/.venv/lib/python3.9/site-packages/kan/MultKAN.py:824: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output n

                                              Признак  Важность (%)
0                                               номер      0.977817
1                                             возраст     22.864311
2                                                 пол      0.331909
3                                          масса_тела      6.720881
4                                                рост      2.443528
5                                          холестерин      3.065411
6                                            нас._жир      1.390817
7                                           натрий_na      0.131970
8                                              сахара      0.890844
9                                             энергия      1.589421
10                                               вода      2.308369
11                                       пищ._волокна      0.570077
12                                     мононенас._жир      0.389859
13                                              

In [ ]:
# Получаем список дочерних модулей модели и предполагаем, что последний – символьный слой
symbolic_layer = list(model.children())[-1]
sym_expr = symbolic_layer.get_symbolic_expression()
print("Символьное представление предсказания:")
print(sym_expr)

AttributeError: 'ModuleList' object has no attribute 'get_symbolic_expression'